In [1]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(URL(
        account='to57169',
        user='yang',
        password='yang123!',
        warehouse = 'fivetran_warehouse',
        database='fivetran',
        schema='bbbnw'

    )
)

In [2]:
master= pd.read_sql_query('DESC TABLE master', engine)
#print(master)

In [3]:
businesslicense = pd.read_sql_query('DESC TABLE businesslicense', engine)
#print(businesslicense)

In [4]:
professional = pd.read_sql_query('DESC TABLE professionallicense', engine)
#print(professional)

In [5]:
active = pd.read_sql_query('DESC TABLE activeorbusinesses', engine)
print(active)

                    name               type    kind null? default primary key  \
0        REGISTRY_NUMBER       NUMBER(38,0)  COLUMN     Y    None           N   
1          BUSINESS_NAME  VARCHAR(16777216)  COLUMN     Y    None           N   
2                ADDRESS  VARCHAR(16777216)  COLUMN     Y    None           N   
3                   CITY  VARCHAR(16777216)  COLUMN     Y    None           N   
4                  STATE  VARCHAR(16777216)  COLUMN     Y    None           N   
5                    ZIP       NUMBER(38,0)  COLUMN     Y    None           N   
6            ENTITY_TYPE  VARCHAR(16777216)  COLUMN     Y    None           N   
7          REGISTRY_DATE  VARCHAR(16777216)  COLUMN     Y    None           N   
8   ASSOCIATED_NAME_TYPE  VARCHAR(16777216)  COLUMN     Y    None           N   
9             FIRST_NAME  VARCHAR(16777216)  COLUMN     Y    None           N   
10           MIDDLE_NAME  VARCHAR(16777216)  COLUMN     Y    None           N   
11     ADDRESS_CONTINUED  VA

In [15]:
#features_to_use = ["BUSINESS","STATE"， "ZIP”]
#Read businessname, state, and zipcode from master and 3rd party dataset, excluding all empty ZIP codes. 
master = pd.read_sql_query('SELECT DISTINCT business, state, left(zip, 5) as zip FROM master WHERE zip IS NOT NULL', engine)
business = pd.read_sql_query('SELECT businessname as business, mailingstatecode as state, left(mailingzip, 5) as zip FROM businesslicense WHERE mailingzip IS NOT NULL', engine)
active_business_df = pd.read_sql_query('SELECT business_name as business, state as state, left(zip, 5) as zip FROM activeorbusinesses WHERE zip IS NOT NULL', engine)
prof_license_df = pd.read_sql_query('SELECT owners as business, state as state, left(zip, 5) as zip FROM professionallicense WHERE zip IS NOT NULL', engine)

In [6]:
#features_to_use = ["BUSINESS","STATE"， "ZIP”]
#Read businessname, state, and zipcode from master and 3rd party dataset
#master = pd.read_sql_query('SELECT business, state, left(zip, 5) as zip FROM master', engine)
#professional = pd.read_sql_query('SELECT * FROM professionallicense', engine)
#business = pd.read_sql_query('SELECT businessname as business, mailingstatecode as state, left(mailingzip, 5) as zip FROM businesslicense', engine)

In [7]:
#print(master.dtypes)
#print(business.dtypes)

In [8]:
print(master.count())

business    368814
state       368814
zip         345447
dtype: int64


In [9]:
print(business.count())


business    74524
state       74524
zip         74524
dtype: int64


In [11]:
#Separate dataframe by state to improve the performance.
#We will write a loop here to process for all states 

list_of_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
                 "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
                 "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"] 

In [12]:
for state in list_of_states:
    master_state =  master[(master.state==state)]
    business_state = business[(business.state==state)] 
    print(state)
    print(master_state['business'].count()), print(business_state['business'].count()), print(master_state['business'].count()*business_state['business'].count())

AL
10
55
550
AK
16963
66196
1122882748
AZ
186
238
44268
AR
15
52
780
CA
730
782
570860
CO
124
311
38564
CT
22
46
1012
DE
11
53
583
FL
130
356
46280
GA
69
127
8763
HI
38
47
1786
ID
41879
181
7580099
IL
111
219
24309
IN
28
85
2380
IA
31
38
1178
KS
21
79
1659
KY
19
53
1007
LA
11
54
594
ME
3
17
51
MD
40
97
3880
MA
33
84
2772
MI
42
146
6132
MN
78
217
16926
MS
3
15
45
MO
50
162
8100
MT
15777
95
1498815
NE
20
54
1080
NV
80
99
7920
NH
2
15
30
NJ
49
171
8379
NM
24
37
888
NY
123
253
31119
NC
47
110
5170
ND
22
9
198
OH
78
175
13650
OK
25
57
1425
OR
99543
385
38324055
PA
68
158
10744
RI
4
5
20
SC
20
45
900
SD
19
23
437
TN
54
124
6696
TX
233
624
145392
UT
129
211
27219
VT
3
8
24
VA
53
184
9752
WA
167937
1546
259630602
WV
9
5
45
WI
50
86
4300
WY
1110
33
36630


In [13]:
from fuzzywuzzy import fuzz

def startEngine(master, thirdparty, filename):
    preprocess3rdparty(thirdparty)
    process3rdparty(master, thirdparty, filename)

# Add assistant fields into thirdparty dataframe
def preprocess3rdparty(thirdparty):
    thirdparty['zip_match_percent'] = pd.Series(0, index=thirdparty.index)
    thirdparty['name_match_percent'] = pd.Series(0, index=thirdparty.index)
    thirdparty['combined_percent'] = pd.Series(0, index=thirdparty.index)
    thirdparty['master_businessname'] = pd.Series("", index=thirdparty.index)
    thirdparty['master_zip'] = pd.Series("", index=thirdparty.index)
    thirdparty['duplicate'] = pd.Series(0, index=thirdparty.index)  

# Process thirdparty
def process3rdparty(master, thirdparty, filename):
    list_of_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
                    "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
                     "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",""]

    thirdparty_aggregate = pd.DataFrame(data=None, columns=thirdparty.columns,index=[0])

    for state in list_of_states:
        master_by_state = master[(master.state==state)]
        thirdparty_by_state = thirdparty[(thirdparty.state==state)] 
        similarityEngine(thirdparty_by_state, master_by_state)
        thirdparty_aggregate = thirdparty_aggregate.append(thirdparty_by_state)

    markDuplicate(thirdparty_aggregate) 
    exportcsv(thirdparty_aggregate, filename)

# Compare and get the similarity
def similarityEngine(thirdparty_temp, master_temp):    
    if(thirdparty_temp['business'].count()*master_temp['business'].count()<100000000):
        computeSimilarity(thirdparty_temp, master_temp)
    else:
        computeSimilarity2(thirdparty_temp, master_temp)

# Combine name similarity with zipcode similarity
def computeSimilarity(thirdparty_temp, master_temp):
    for index,row in thirdparty_temp.iterrows():
        name_similarity = 0
        zip_similarity = 0
        max_combine = 0
        master_name = "xxx"
        master_zip = 0
        #business_name = thirdparty_temp.get_value(index, 'business')
        #business_zip = thirdparty_temp.get_value(index, 'zip')
        business_name = row['business']
        business_zip = row['zip']

        for index1,row1 in master_temp.iterrows():
            business = row1['business']
            zipcode = row1['zip']
            name_similar = fuzz.ratio(business_name.lower(), business.lower())
            if(name_similar >= 90):
                name_similarity = name_similar
                max_combine = name_similar
                master_name = business
                master_zip = zipcode
                break;
            elif(name_similar < 75):
                continue;
            elif(name_similar>=75 and name_similar<90):
                zip_similar = fuzz.ratio(business_zip, zipcode)

                if(zip_similar<40):
                    combine_similar = name_similar
                else:
                    combine_similar = name_similar * 0.8 + zip_similar * 0.2

                if(combine_similar > max_combine):
                    name_similarity = name_similar
                    zip_similarity = zip_similar
                    max_combine = combine_similar
                    master_name = business
                    master_zip = zipcode
        
        thirdparty_temp.set_value(index, 'name_match_percent', name_similarity)
        thirdparty_temp.set_value(index, 'zip_match_percent', zip_similarity)
        thirdparty_temp.set_value(index, 'combined_percent', max_combine)
        thirdparty_temp.set_value(index, 'master_businessname', master_name)
        thirdparty_temp.set_value(index, 'master_zip', master_zip)

# Use only name similarity to improve performance
def computeSimilarity2(thirdparty_temp, master_temp):
    for index,row in thirdparty_temp.iterrows():
        name_similarity = 0
        zip_similarity = 0
        max_combine = 0
        master_zip = 0
        master_name = 'xxx'
        business_name = row['business']
        business_zip = row['zip']
        
        master_temp_zip = master_temp[(master_temp.zip==business_zip)]
        
        for index1, row1 in master_temp_zip.iterrows():
            business = row1['business']
            name_similar = fuzz.ratio(business_name.lower(), business.lower())
            if(name_similar>=90):
                name_similarity = name_similar
                max_combine = name_similar
                master_name = business
                break;
            elif(name_similar<75):
                #not duplicate
                continue;
            elif(name_similar>=75 and name_similar<90):
                combine_similar = name_similar
                if(combine_similar > max_combine):
                    name_similarity = name_similar
                    max_combine = name_similar
                    master_name = business
        
        thirdparty_temp.set_value(index, 'name_match_percent', name_similarity)
        thirdparty_temp.set_value(index, 'zip_match_percent', zip_similarity)
        thirdparty_temp.set_value(index, 'combined_percent', max_combine)
        thirdparty_temp.set_value(index, 'master_businessname', master_name)
        thirdparty_temp.set_value(index, 'master_zip', master_zip)

# mark duplicate indicator
def markDuplicate(thirdparty):
    for index, row in thirdparty.iterrows():
        if(row['combined_percent']>85):
            if(row['zip'] == row['master_zip']):
                thirdparty.set_value(index, 'duplicate', 1)
            else:
                thirdparty.set_value(index, 'duplicate', 2)
        elif(row['combined_percent']<75):
            thirdparty.set_value(index, 'duplicate', 0)
        elif(row['combined_percent']>=75 and row['combined_percent']<=85):
            thirdparty.set_value(index, 'duplicate', 2)

#export result      
def exportcsv(thirdparty, filename):
    thirdparty_duplicate = thirdparty[(thirdparty.duplicate == 1)]
    thirdparty_non_duplicate = thirdparty[(thirdparty.duplicate == 0)]
    thirdparty_manual = thirdparty[((thirdparty.duplicate == 2))]
    
    thirdparty_duplicate.to_csv(filename+"_duplicate.csv")
    thirdparty_non_duplicate.to_csv(filename+"_noduplicate.csv")
    thirdparty_manual.to_csv(filename+"_manual.csv")

c:\program files (x86)\python36-32\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [14]:
startEngine(master, business, "business")

KeyboardInterrupt: 